In [2]:
import torch
import time
import platform
import sys

def detect_gpus():
    """检测并返回可用的 GPU 设备"""
    print(f"Python 版本: {platform.python_version()}")
    print(f"PyTorch 版本: {torch.__version__}")
    print(f"系统信息: {platform.system()} {platform.release()}")
    
    available_devices = []
    
    # 检测 CUDA (NVIDIA GPU)
    if torch.cuda.is_available():
        cuda_device = torch.device("cuda")
        available_devices.append(cuda_device)
        print("\n== NVIDIA CUDA 可用 ==")
        print(f"CUDA 版本: {torch.version.cuda}")
        print(f"GPU 数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"  设备 {i}: {torch.cuda.get_device_name(i)}")
            props = torch.cuda.get_device_properties(i)
            print(f"    计算能力: {props.major}.{props.minor}")
            print(f"    总显存: {props.total_memory / 1024**3:.2f} GB")
    else:
        print("\n== NVIDIA CUDA 不可用 ==")
    
    # 检测 MPS (Apple Silicon GPU)
    if hasattr(torch, 'mps') and hasattr(torch.mps, 'is_available'):
        if torch.mps.is_available():
            mps_device = torch.device("mps")
            available_devices.append(mps_device)
            print("\n== Apple MPS 可用 ==")
            print("注: Apple Silicon GPU 不提供详细硬件信息接口")
        else:
            print("\n== Apple MPS 不可用 ==")
    else:
        print("\n== 当前 PyTorch 不支持 Apple MPS ==")
    
    # 检测 ROCm (AMD GPU) - 通过间接方式
    try:
        if hasattr(torch, 'hip') and torch.hip.is_available():
            hip_device = torch.device("hip")
            available_devices.append(hip_device)
            print("\n== AMD ROCm (HIP) 可用 ==")
        else:
            print("\n== AMD ROCm (HIP) 不可用 ==")
    except:
        print("\n== 当前 PyTorch 不支持 AMD ROCm ==")
    
    # 始终有 CPU
    cpu_device = torch.device("cpu")
    available_devices.append(cpu_device)
    
    return available_devices

def benchmark_device(device, size=5000):
    """对指定设备进行矩阵乘法性能测试"""
    print(f"\n在 {device} 上测试 {size}x{size} 矩阵乘法性能...")
    
    # 创建 CPU 张量
    a_cpu = torch.randn(size, size)
    b_cpu = torch.randn(size, size)
    
    # 转移到目标设备
    try:
        a = a_cpu.to(device)
        b = b_cpu.to(device)
        
        # 预热
        _ = torch.matmul(a, b)
        
        # 计时
        start = time.time()
        c = torch.matmul(a, b)
        
        # 确保完成计算
        if device.type == "cuda":
            torch.cuda.synchronize()
        elif device.type == "mps":
            torch.mps.synchronize()
        elif hasattr(torch, 'hip') and device.type == "hip":
            torch.hip.synchronize()
        
        # 如果是 GPU，转回 CPU 以确保操作完成
        if device.type != "cpu":
            _ = c.to("cpu")
            
        duration = time.time() - start
        print(f"{device} 计算时间: {duration:.4f} 秒")
        return duration
    except Exception as e:
        print(f"在 {device} 上测试失败: {e}")
        return None

# 主函数
def main():
    print("=" * 60)
    print("PyTorch GPU 检测与性能测试")
    print("=" * 60)
    
    # 检测可用设备
    available_devices = detect_gpus()
    
    if len(available_devices) <= 1:  # 只有 CPU
        print("\n没有检测到可用的 GPU 设备，仅有 CPU 可用。")
        benchmark_device(torch.device("cpu"), size=2000)
        return
    
    # 为每个设备运行基准测试
    cpu_time = benchmark_device(torch.device("cpu"), size=2000)
    
    for device in available_devices:
        if device.type != "cpu":
            gpu_time = benchmark_device(device, size=2000)
            if cpu_time and gpu_time:
                print(f"{device} 加速比: {cpu_time/gpu_time:.2f}x")

# 运行主函数
if __name__ == "__main__":
    main()

PyTorch GPU 检测与性能测试
Python 版本: 3.11.11
PyTorch 版本: 2.0.1
系统信息: Darwin 23.6.0

== NVIDIA CUDA 不可用 ==

== 当前 PyTorch 不支持 Apple MPS ==

== AMD ROCm (HIP) 不可用 ==

没有检测到可用的 GPU 设备，仅有 CPU 可用。

在 cpu 上测试 2000x2000 矩阵乘法性能...
cpu 计算时间: 0.3588 秒


In [4]:
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# 定义 Transformer 模型
class TransformerModel(nn.Module):
    def __init__(self, input_dim, nhead, hidden_dim, num_layers, output_dim):
        """
        初始化 Transformer 模型

        参数:
        input_dim (int): 输入特征维度
        nhead (int): 多头注意力头数
        hidden_dim (int): 前馈网络的隐藏层维度
        num_layers (int): Transformer 编码层数
        output_dim (int): 输出维度
        """
        super(TransformerModel, self).__init__()
        # 定义一个 Transformer 编码层
        encoder_layer = TransformerEncoderLayer(d_model=input_dim, nhead=nhead, dim_feedforward=hidden_dim)
        # 将多个编码层堆叠成 Transformer 编码器
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_layers)
        # 定义一个全连接层，将 Transformer 的输出映射到最终的输出维度
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        """
        前向传播函数

        参数:
        x (torch.Tensor): 输入张量，形状为 (seq_len, batch_size, input_dim)

        返回:
        torch.Tensor: 输出张量，形状为 (batch_size, output_dim)
        """
        # 通过 Transformer 编码器处理输入
        out = self.transformer(x)  # (seq_len, batch_size, input_dim)
        # 对序列维度求平均，聚合序列信息
        out = self.fc(out.mean(dim=0))  # (batch_size, input_dim) -> (batch_size, output_dim)
        return out

# 模型参数
input_dim = 10  # 输入特征维度
nhead = 2  # 多头注意力头数
hidden_dim = 50  # 前馈网络的隐藏层维度
num_layers = 2  # Transformer 编码层数
output_dim = 1  # 输出维度

# 构建模型
model = TransformerModel(input_dim=input_dim, nhead=nhead, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim)

# 示例输入数据：seq_len=5, batch_size=32
x = torch.randn(5, 32, input_dim)  # 注意输入的形状 (seq_len, batch_size, input_dim)
output = model(x)

print("Transformer 输出维度:", output.shape)  # (batch_size, output_dim)

Transformer 输出维度: torch.Size([32, 1])


In [5]:
import torch
import torch.nn as nn

# 定义 RNN 模型
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        初始化 RNN 模型

        参数:
        input_dim (int): 每个时间步的输入特征维度
        hidden_dim (int): 隐层维度
        output_dim (int): 输出维度（如回归任务）
        """
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """
        前向传播函数

        参数:
        x (torch.Tensor): 输入张量，形状为 (batch_size, seq_len, input_dim)

        返回:
        torch.Tensor: 输出张量，形状为 (batch_size, output_dim)
        """
        out, _ = self.rnn(x)  # (batch_size, seq_len, hidden_dim)
        out = self.fc(out[:, -1, :])  # 提取最后一个时间步的输出
        return out

# 模型参数
input_dim = 10  # 每个时间步的输入特征维度
hidden_dim = 20  # 隐层维度
output_dim = 1  # 输出维度（如回归任务）

# 构建模型
model = RNNModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

# 示例输入：batch_size=32, seq_len=5
# 32个样本，每个样本有5个时间步，每个时间步有10个特征
x = torch.randn(32, 5, input_dim)

# model(x)  # 前向传播
# 运行模型
output = model(x)

# 注意：RNN 的输入形状为 (batch_size, seq_len, input_dim)
# 输出形状为 (batch_size, output_dim)
print("RNN 输出维度:", output.shape)  # (batch_size, output_dim)

RNN 输出维度: torch.Size([32, 1])


In [1]:
import torch
print(torch.__version__)

# import tensorflow as tf
# print(tf.__version__)

2.0.1
